Author: Elisha Paine

In [ ]:
!git clone https://github.com/NVlabs/stylegan3.git

Cloning into 'stylegan3'...
remote: Enumerating objects: 207, done.
remote: Total 207 (delta 0), reused 0 (delta 0), pack-reused 207
Receiving objects: 100% (207/207), 4.17 MiB | 3.11 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [ ]:
# fix an issue with the stylegan3 repo
file = "stylegan3/torch_utils/ops/grid_sample_gradfix.py"
with open(file) as f:
  lines = f.read().split("\n")
lines[59] = "        op, _ = torch._C._jit_get_operation('aten::grid_sampler_2d_backward')"
with open(file, "w") as f:
  f.write("\n".join(lines))

In [ ]:
!pip install ninja
!pip uninstall jax jaxlib -y
!pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 108 kB 37.3 MB/s 
Found existing installation: jax 0.3.14
Uninstalling jax-0.3.14:
  Successfully uninstalled jax-0.3.14
Found existing installation: jaxlib 0.3.14+cuda11.cudnn805
Uninstalling jaxlib-0.3.14+cuda11.cudnn805:
  Successfully uninstalled jaxlib-0.3.14+cuda11.cudnn805
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     |████████████████████████████████| 939 kB 29.1 MB/s 
     |████████████████████████████████| 175.7 MB 7.8 kB/s 
  Created wheel for jax: filename=jax-0.3.10-py3-none-any.whl size=1088066 sha256=d4ddf0686b762a76b6e81b879cc6d32398ea2368e7ea0fb9c50c700020614735
  Stored in directory: /root/.cache/pip/wheels/1c/05/23/36730377cd7311156f1e5eb5e7c683d5cdac1654658c095cc5
Successfully built jax


In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
!python stylegan3/train.py --outdir=weights --cfg=stylegan3-t --data=drive/MyDrive/Colab\ Notebooks/os_tiles_256 --batch=16 --gpus=1 --gamma=8


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 32768,
    "channel_max": 512,
    "magnitude_ema_beta": 0.9994456359721023
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 8.0
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "prefetch_fact

In [ ]:
import cv2
import numpy
import pickle
import sys
import torch
sys.path.append("stylegan3")
import torch_utils.persistence
with open("weights/00008-stylegan3-t-tiles-gpus1-batch8-gamma8/network-snapshot-000000.pkl", "rb") as f:
  G = pickle.load(f)["G_ema"].cuda()
z = torch.randn([1, G.z_dim]).cuda()
img = G(z, None)
cv_im = img.cpu().numpy()
cv_im *= 2 ** 8
w, h = cv_im.shape[2:]
cv_im = cv_im.squeeze(0).transpose((1, 2, 0)).astype(numpy.uint8, casting="unsafe")
cv2.imwrite("sample.png", cv_im);

FileNotFoundError: ignored